In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split



In [ ]:
def create_lag_features(df, lag_features, lags):
    for lag in lags:
        for feature in lag_features:
            df[f"{feature}_lag{lag}"] = df.groupby('series_id')[feature].shift(lag)
    return df.dropna()

lag_features = ["anglez", "enmo"]
lags = [1, 2, 3]

df = create_lag_features(df, lag_features, lags)

features = [f"{feature}_lag{lag}" for feature in lag_features for lag in lags]




In [ ]:
def train_test_split_time_series(df):
    train_dfs = []
    test_dfs = []
    for _, group in df.groupby('series_id'):
        train_size = int(len(group) * 0.8)
        train_dfs.append(group.iloc[:train_size])
        test_dfs.append(group.iloc[train_size:])
    return pd.concat(train_dfs), pd.concat(test_dfs)

train_df, test_df = train_test_split_time_series(df)



In [ ]:
xgb_regressor = xgb.XGBRegressor(objective ='reg:squarederror')
X_train = train_df[features]
y_train = train_df["anglez"]
xgb_regressor.fit(X_train, y_train)
